In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Dataset/airfoil_self_noise.csv",sep=";")
data.head(5)



,frequency,angle_of_attack,chord_length,free_stream_velocity,suction_side_displacement_thickness,scaled_sound_pressure_level
0,800,0.0,0.3048,71.3,0.002663,126.201
1,1000,0.0,0.3048,71.3,0.002663,125.201
2,1250,0.0,0.3048,71.3,0.002663,125.951
3,1600,0.0,0.3048,71.3,0.002663,127.591
4,2000,0.0,0.3048,71.3,0.002663,127.461


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, var_red=None, value=None):
        ''' constructor '''

        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.var_red = var_red

        # for leaf node
        self.value = value

In [ ]:
class DecisionTreeRegressor():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''

        # initialize the root of the tree
        self.root = None

        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth

    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree '''

        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        best_split = {}
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["var_red"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"],
                            left_subtree, right_subtree, best_split["var_red"])

        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)

    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''

        # dictionary to store the best split
        best_split = {}
        max_var_red = -float("inf")
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_var_red = self.variance_reduction(y, left_y, right_y)
                    # update the best split if needed
                    if curr_var_red>max_var_red:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["var_red"] = curr_var_red
                        max_var_red = curr_var_red

        # return best split
        return best_split

    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''

        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right

    def variance_reduction(self, parent, l_child, r_child):
        ''' function to compute variance reduction '''

        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        reduction = np.var(parent) - (weight_l * np.var(l_child) + weight_r * np.var(r_child))
        return reduction

    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''

        val = np.mean(Y)
        return val

    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''

        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.var_red)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)

    def fit(self, X, Y):
        ''' function to train the tree '''

        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)

    def make_prediction(self, x, tree):
        ''' function to predict new dataset '''

        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

    def predict(self, X):
        ''' function to predict a single data point '''

        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions

In [ ]:
class GradientBoostingRegressor:
  def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3, min_samples_split=2):

    # n_estimators is the no of trees that will be added for boosting
    #learning rate is the value by which predictions of each tree are scaled
    #max_depth is the maximum depth of a regression tree
    #min no of samples required to split a node

    self.n_estimators = n_estimators
    self.learning_rate = learning_rate
    self.max_depth = max_depth
    self.min_samples_split = min_samples_split
    self.trees = []
    self.initial_prediction = None

  def fit(self,X,Y):
    #Initally we will have a tree with one leaf that predict as the mean of the target values
    self.initial_prediction = np.mean(Y)
    predictions = np.full(Y.shape, self.initial_prediction)


    #Building trees that would contribute to the improvement in the current residuals
    for _ in range(self.n_estimators):
      residuals = Y - predictions

      tree = DecisionTreeRegressor(min_samples_split=self.min_samples_split, max_depth = self.max_depth)
      #training the new tree on the residuals , so that It learns to reduce it
      tree.fit(X, residuals.reshape(-1,1))


      predictions_tree = np.array(tree.predict(X)).reshape(-1,1)

      # Improved prediciton = Initial prediciton + lr * residual from new tree
      predictions = predictions + self.learning_rate * predictions_tree.flatten()


      self.trees.append(tree)


  def predict(self, X):
    predictions = np.full((X.shape[0],) ,self.initial_prediction)

    #Adding the scaled predciton of all the trees in the esemble

    for tree in self.trees:
      predictions += self.learning_rate * np.array(tree.predict(X))
    return predictions



In [ ]:
# Example usage
# Assuming 'data' is your dataset as a pandas DataFrame
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=41)

regressor = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)
regressor.fit(X_train, Y_train)
Y_pred = regressor.predict(X_test)
test_loss = mean_squared_error(Y_test, Y_pred)
print(f"Test MSE: {test_loss}")


Test MSE: 4.464907582981265


In [ ]:
import numpy as np

class GradientBoostingClassifier:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3, min_samples_split=2):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.trees = []

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def fit(self, X, y):
        # Initial prediction (log odds)
        initial_pred = np.log(np.mean(y) / (1 - np.mean(y)))
        self.initial_prediction = initial_pred
        predictions = np.full(y.shape, initial_pred)

        # Boosting process
        for _ in range(self.n_estimators):
            # Calculate pseudo-residuals (negative gradient of log-loss)
            probas = self.sigmoid(predictions)
            residuals = y - probas

            # Fit a decision tree on the residuals
            tree = DecisionTreeRegressor(min_samples_split=self.min_samples_split, max_depth=self.max_depth)
            # The decision tree expects a 2D array, so we reshape residuals
            tree.fit(X, residuals.reshape(-1, 1))
            # Update predictions with the new tree
            predictions += self.learning_rate * np.array(tree.predict(X)).flatten()

            # Add the tree to the ensemble
            self.trees.append(tree)

    def predict_proba(self, X):
        # Start with the initial prediction
        predictions = np.full((X.shape[0],), self.initial_prediction)

        # Sum the contributions of each tree
        for tree in self.trees:
            predictions += self.learning_rate * np.array(tree.predict(X)).flatten()

        # Apply the sigmoid function to get probabilities
        return self.sigmoid(predictions)

    def predict(self, X):
        # Convert probabilities to binary class labels
        probas = self.predict_proba(X)
        return np.where(probas >= 0.5, 1, 0)


In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

X, y = make_classification(n_samples=50, n_features=4, n_informative=2,
                           n_redundant=0, n_clusters_per_class=1,
                           flip_y=0, random_state=41)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy}")

Test Accuracy: 0.8
